In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

In [2]:
df_all = pd.read_excel('data.xlsx')
col2drop = ['NO','Name','SequenceAmount','Sequence','Sites Number','Binding Sites']
X = df_all.drop(col2drop,axis=1)
y = df_all['Binding Sites']  
# X.head()

In [3]:
#data normalization
names = X.columns
X = normalize(X, norm='l2')
X = pd.DataFrame(X, columns=names)
X.head()

,v1*seqloc,v2*seqloc,v3*seqloc,v4*seqloc,cb*seqloc,cp(rs)*loc,cp(hb)*loc,cp(fg)*loc,v1*energyloc,v2*energyloc,v3*energyloc,v4*energyloc,cp(rs)*energyloc,cp(hb)*energyloc,cp(fg)*energyloc,cb*energyloc
0,0.055721,0.676584,0.000000,-0.131766,0.065883,0.021961,0.000000,0.000000,0.057642,0.699914,0.000000,-0.136310,0.022718,0.000000,0.000000,0.068155
1,0.055721,0.676584,0.000000,-0.131766,0.065883,0.021961,0.000000,0.000000,0.057642,0.699914,0.000000,-0.136310,0.022718,0.000000,0.000000,0.068155
2,0.028761,0.484852,0.496547,-0.016483,0.015405,0.005135,0.005135,0.005135,0.029753,0.501572,0.513669,-0.017051,0.005312,0.005312,0.005312,0.015936
3,0.028761,0.484852,0.496547,-0.016483,0.015405,0.005135,0.005135,0.005135,0.029753,0.501572,0.513669,-0.017051,0.005312,0.005312,0.005312,0.015936
4,0.019112,0.340836,0.605266,-0.009948,0.006545,0.003272,0.003272,0.003272,0.019772,0.352589,0.626138,-0.010291,0.003385,0.003385,0.003385,0.006771


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
X_train_isbs = X_train.loc[y_train == 1]
X_train_notbs = X_train.loc[y_train == 0]
# X_train_isbs.head()

In [6]:
miu_isbs = X_train_isbs.describe().loc['mean']
miu_notbs = X_train_notbs.describe().loc['mean']
# miu_isbs

In [7]:
sum_isbs = np.zeros([16,16])
p_train_isbs = X_train_isbs.shape[0]
for i in range(16):
    for j in range(16):
        ti = X_train_isbs.iloc[:,i].to_numpy()
        miui = np.full((p_train_isbs),miu_isbs.iat[i])
        tj = X_train_isbs.iloc[:,j].to_numpy()
        miuj = np.full((p_train_isbs),miu_isbs.iat[j])
        sum_isbs[i][j] = np.mean((ti - miui)*(tj - miuj))
# sum_isbs

In [8]:
sum_notbs = np.zeros([16,16])
p_train_notbs = X_train_notbs.shape[0]
for i in range(16):
    for j in range(16):
        ti = X_train_notbs.iloc[:,i].to_numpy()
        miui = np.full((p_train_notbs),miu_notbs.iat[i])
        tj = X_train_notbs.iloc[:,j].to_numpy()
        miuj = np.full((p_train_notbs),miu_notbs.iat[j])
        sum_notbs[i][j] = np.mean((ti - miui)*(tj - miuj))
# sum_notbs

In [9]:
delta_isbs = []
p_test = X_test.shape[0]
for i in range(p_test):
    tmp = (X_test.iloc[i] - miu_isbs).to_numpy()
    tmp_t = np.transpose(tmp)
    sum_isbs_inv = np.linalg.inv(sum_isbs)
    delta_isbs.append(tmp_t.dot(sum_isbs_inv).dot(tmp))
# len(delta_isbs)



In [10]:
delta_notbs = []
for i in range(p_test):
    tmp = (X_test.iloc[i] - miu_notbs).to_numpy()
    tmp_t = np.transpose(tmp)
    sum_notbs_inv = np.linalg.inv(sum_notbs)
    delta_notbs.append(tmp_t.dot(sum_notbs_inv).dot(tmp))
# len(delta_notbs)

In [11]:
def sigmoid_pos(x):
    s = 1 / (1 + np.exp(-x)) + 1
    return s

In [12]:
lnp_train_isbs = np.full((p_test),sigmoid_pos(p_train_isbs))
lnsum_isbs_det = np.full((p_test),sigmoid_pos(np.linalg.det(sum_isbs)))
delta_isbs = np.array(delta_isbs)
eta_isbs = lnp_train_isbs - delta_isbs/2 - lnsum_isbs_det/2
# len(eta_isbs)

In [13]:
lnp_train_notbs = np.full((p_test),sigmoid_pos(p_train_notbs))
lnsum_notbs_det = np.full((p_test),sigmoid_pos(np.linalg.det(sum_notbs)))
delta_notbs = np.array(delta_notbs)
eta_notbs = lnp_train_notbs - delta_notbs/2 - lnsum_notbs_det/2
# len(eta_notbs)

In [14]:
y_pred = eta_isbs > eta_notbs
# y_pred

In [15]:
accuracy_score(y_test, y_pred)

0.512779552715655